In [ ]:
#| default_exp xml

# XML

> Concise generation of XML.

In [ ]:
#| export
from fastcore.utils import *

import types,json

from dataclasses import dataclass, asdict
from typing import Mapping
from functools import partial
from html import escape

In [ ]:
from IPython.display import Markdown
from pprint import pprint

from fastcore.test import test_eq

## FT functions

In [ ]:
#|export
def _fix_k(k): return k if k=='_' else k.lstrip('_').replace('_', '-')

The `_fix_k()` function takes one parameter `k`, which is expected to be a string representing an attribute key.

**Explanation**
`return k if k=='_' else k.lstrip('_').replace('_', '-')` 
   - If `k` equals exactly `'_'`, it returns `k` unchanged
   - Otherwise, it performs two operations on `k`:
     - First, `k.lstrip('_')` removes any leading underscores from the string
     - Then, `.replace('_', '-')` replaces any remaining underscores with hyphens

>This function is used to transform attribute names from Python-friendly formats to HTML/XML-friendly formats. 

For example:
- `_fix_k('_class')` would return `'class'` (removes leading underscore)
- `_fix_k('data_value')` would return `'data-value'` (replaces inner underscore with hyphen)
- `_fix_k('_my_attr')` would return `'my-attr'` (removes leading underscore and replaces remaining ones)
- `_fix_k('_')` would return `'_'` (special case, preserved as-is)

**Special Case(`_`)**

The special case where `k == '_'` returns `'_'` unchanged is interesting and serves a specific purpose.

In HTML/XML, there are some attributes that might need to be represented with an underscore. While most attributes follow standard naming conventions, there might be custom attributes or specific use cases where an underscore is needed as the actual attribute name.

This special case allows users of the library to explicitly specify an underscore attribute by using `'_'` directly. Without this exception, the function would strip the underscore and return an empty string, which wouldn't be a valid attribute name.

>For example, if someone needed to create a custom XML element with an attribute literally named `_`, they could do so with this library without the function interfering with that intent.

It's a relatively rare edge case, but it ensures the library can handle all valid XML attribute scenarios, including those that might use underscores as part of a custom namespace or non-standard attribute naming convention.

In [ ]:
#| export
_specials = set('@.-!~:[](){}$%^&*+=|/?<>,`')

`_specials` is defined as a global variable that contains a set of special characters.

This set is used in the `attrmap()` function (see next cell) to determine whether an attribute name contains the special characters mentioned.

This is important because HTML5 and XML allow custom data attributes (like `data-*` attributes) and other attributes that might contain these special characters, which should be preserved exactly as provided by the user.

In [ ]:
#| export
def attrmap(o):
    if _specials & set(o): return o
    o = dict(htmlClass='class', cls='class', _class='class', klass='class',
             _for='for', fr='for', htmlFor='for').get(o, o)
    return _fix_k(o)

`attrmap` function takes one parameter `o` (which is expected to be a string representing an attribute name).

1. `if _specials & set(o): return o`
   - Converts the input string `o` to a set of characters using `set(o)`
   - Uses the `&` operator to find the intersection with the `_specials` set
   - If there's any overlap (meaning `o` contains any special characters), it returns `o` unchanged
   - This preserves attribute names that contain special characters

2. `o = dict(htmlClass='class', cls='class', _class='class', klass='class', _for='for', fr='for', htmlFor='for').get(o, o)`
   - Creates a dictionary mapping various alternative names to standard HTML attribute names
   - Uses `.get(o, o)` to look up the input `o` in this dictionary
   - If `o` is found in the dictionary, returns the corresponding value
   - If `o` is not found, returns `o` itself (the second argument to `.get()`)
   - This handles common Python workarounds for reserved keywords like `class` and `for`

3. `return _fix_k(o)`
   - Passes the result to the `_fix_k()` function above
   - This will strip leading underscores and replace remaining underscores with hyphens
   - Returns the fully transformed attribute name

The function serves two main purposes:
1. It preserves attribute names with special characters
2. It handles common alternative names for HTML attributes that conflict with Python keywords

For example:
- `attrmap('data-value')` returns `'data-value'` (contains `-`, a special character)
- `attrmap('cls')` returns `'class'` (maps Python-friendly alternative to HTML standard)
- `attrmap('_style')` returns `'style'` (removes leading underscore via `_fix_k`)
- `attrmap('background_color')` returns `'background-color'` (replaces `_` with `-` via `_fix_k`)

This function is crucial for the module's usability, as it allows developers to use Python-friendly attribute names that get properly converted to standard HTML/XML attributes or custom attributes with special characters.

> Possible python substitutes for attributes `class` and `for` (since these are reserved keywords in python):
>   1. `class` : cls, _class, htmlClass, klass
>   2. `for`: fr, _for, htmlFor

In [ ]:
#| export
def valmap(o):
    if is_listy(o): return ' '.join(map(str,o)) if o else None
    if isinstance(o, dict): return '; '.join(f"{k}:{v}" for k,v in o.items()) if o else None
    return o

The function `valmap` takes one parameter `o` (representing an attribute value that needs to be converted to an appropriate string format for HTML/XML).

1. `if is_listy(o): return ' '.join(map(str,o)) if o else None` - This line:
   - Checks if `o` is a list-like object using the `is_listy()` function (which is imported from fastcore.utils)
   - If `o` is list-like and not empty (`if o`), it:
     - Converts each item in the list to a string using `map(str, o)`
     - Joins those strings with spaces using `' '.join()`
     - For example, `valmap(['btn', 'primary'])` would return `'btn primary'` (useful for CSS classes)
   - If `o` is list-like but empty, it returns `None`

2. `if isinstance(o, dict): return '; '.join(f"{k}:{v}" for k,v in o.items()) if o else None` - This line:
   - Checks if `o` is a dictionary using `isinstance(o, dict)`
   - If `o` is a dictionary and not empty, it:
     - Creates a string for each key-value pair in the format `"key:value"`
     - Joins these strings with semicolons using `'; '.join()`
     - For example, `valmap({'margin': '10px', 'padding': '5px'})` would return `'margin:10px; padding:5px'` (useful for inline CSS)
   - If `o` is an empty dictionary, it returns `None`

3. `return o` - If `o` is neither list-like nor a dictionary, it returns `o` unchanged.

The function's purpose is to convert complex Python data structures (lists and dictionaries) into string formats that are appropriate for HTML/XML attributes:
- Lists become space-separated values (ideal for CSS classes)
- Dictionaries become semicolon-separated key-value pairs (ideal for inline styles)
- Other values pass through unchanged

This function is particularly useful for:
- CSS classes: `valmap(['btn', 'primary'])` → `'btn primary'`
- Inline styles: `valmap({'color': 'red', 'font-size': '12px'})` → `'color:red; font-size:12px'`
- Empty collections: Both empty lists and empty dictionaries become `None`, which will cause the attribute to be omitted

This makes the module more intuitive to use, as developers can pass Python data structures directly as attribute values.

In [ ]:
#| export
def _flatten_tuple(tup):
    if not any(isinstance(item, tuple) for item in tup): return tup
    result = []
    for item in tup:
        if isinstance(item, tuple): result.extend(item)
        else: result.append(item)
    return tuple(result)

1. `def _flatten_tuple(tup):` - Defines a function named `_flatten_tuple` that takes one parameter `tup` (which is expected to be a tuple, potentially containing nested tuples).

2. `if not any(isinstance(item, tuple) for item in tup): return tup` - This line:
   - Uses a generator expression to check if any item in `tup` is itself a tuple
   - If none of the items are tuples (meaning there's no nesting), it returns `tup` unchanged
   - This is an optimization that avoids unnecessary processing for already-flat tuples

3. `result = []` - Initializes an empty list to collect the flattened items.

4. `for item in tup:` - Iterates through each item in the input tuple.

5. `if isinstance(item, tuple): result.extend(item)` - If the current item is a tuple:
   - Uses `result.extend(item)` to add all elements from that tuple to the result list
   - This effectively "unpacks" the nested tuple into the result

6. `else: result.append(item)` - If the current item is not a tuple:
   - Adds it directly to the result list with `result.append(item)`

7. `return tuple(result)` - Converts the final result list back to a tuple and returns it.

The function's purpose is to flatten one level of nesting in a tuple. For example:
- `_flatten_tuple((1, 2, 3))` returns `(1, 2, 3)` (already flat, no change)
- `_flatten_tuple((1, (2, 3), 4))` returns `(1, 2, 3, 4)` (flattens the nested tuple)
- `_flatten_tuple((1, (2, (3, 4)), 5))` returns `(1, 2, (3, 4), 5)` (only flattens one level)

This function is important in the library because it allows users to provide nested structures as arguments, which get flattened appropriately. This makes the API more flexible and intuitive, as users can group related elements together in nested tuples if they wish.

In [ ]:
#|export
def _preproc(c, kw, attrmap=attrmap, valmap=valmap):
    if len(c)==1 and isinstance(c[0], (types.GeneratorType, map, filter)): c = tuple(c[0])
    attrs = {attrmap(k.lower()):valmap(v) for k,v in kw.items() if v is not None}
    return _flatten_tuple(c),attrs

1. `def _preproc(c, kw, attrmap=attrmap, valmap=valmap):` - Defines a function named `_preproc` that takes four parameters:
   - `c`: A tuple of children elements
   - `kw`: A dictionary of keyword arguments (attributes)
   - `attrmap`: A function to map attribute names (defaults to the `attrmap` function we saw earlier)
   - `valmap`: A function to map attribute values (defaults to the `valmap` function we saw earlier)

2. `if len(c)==1 and isinstance(c[0], (types.GeneratorType, map, filter)): c = tuple(c[0])` - This line:
   - Checks if `c` contains exactly one item and that item is a generator, map, or filter object
   - If so, it converts that generator/map/filter to a tuple using `tuple(c[0])`
   - This allows users to pass generators as children, which get automatically expanded
   - For example: `Div(x for x in range(3))` would be expanded to `Div(0, 1, 2)`

3. `attrs = {attrmap(k.lower()):valmap(v) for k,v in kw.items() if v is not None}` - This line:
   - Creates a dictionary comprehension that processes each key-value pair in `kw`
   - Filters out pairs where the value is `None` with `if v is not None`
   - For each remaining pair:
     - Converts the key to lowercase with `k.lower()`
     - Applies the `attrmap` function to the key
     - Applies the `valmap` function to the value
   - The result is a processed dictionary of attributes

4. `return _flatten_tuple(c),attrs` - Finally, it:
   - Applies the `_flatten_tuple` function to the children tuple `c`
   - Returns a tuple containing the flattened children and the processed attributes

The function's purpose is to preprocess both the children elements and attributes before they're used to create an XML/HTML element:

1. It handles generator expressions by expanding them into tuples
2. It flattens nested tuples of children
3. It transforms attribute names using `attrmap`
4. It transforms attribute values using `valmap`
5. It filters out attributes with `None` values

This preprocessing makes the library more flexible and user-friendly by:
- Supporting generators as a concise way to create multiple children
- Allowing nested structures that get automatically flattened
- Converting Python-friendly attribute names to HTML-compatible ones
- Converting Python data structures (lists, dicts) to appropriate string formats for attributes
- Automatically removing attributes with `None` values (a common pattern for conditional attributes)

The function is a key part of the library's preprocessing pipeline that makes the API both powerful and intuitive to use.

In [ ]:
#|export
class FT:
    "A 'Fast Tag' structure, containing `tag`,`children`,and `attrs`"
    def __init__(self, tag:str, cs:tuple, attrs:dict=None, void_=False, **kwargs):
        assert isinstance(cs, tuple)
        self.tag,self.children,self.attrs,self.void_ = tag,cs,attrs,void_
        self.listeners_ = []
    
    def on(self, f): self.listeners_.append(f)
    def changed(self):
        [f(self) for f in self.listeners_]
        return self

    def __setattr__(self, k, v):
        if len(k)>1 and k.startswith('__') or k[-1]=='_' or k in ('tag','children','attrs','void_'): return super().__setattr__(k,v)
        self.attrs[_fix_k(k)] = v
        self.changed()

    def __getattr__(self, k):
        if k.startswith('__'): raise AttributeError(k)
        return self.get(k)

    @property
    def list(self): return [self.tag,self.children,self.attrs]
    def get(self, k, default=None): return self.attrs.get(_fix_k(k), default)
    
    def __repr__(self): return f'{self.tag}({self.children},{self.attrs})'
    def __iter__(self): return iter(self.children)
    def __getitem__(self, idx): return self.children[idx]
    
    def __setitem__(self, i, o):
        self.children = self.children[:i] + (o,) + self.children[i+1:]
        self.changed()

    def __call__(self, *c, **kw):
        c,kw = _preproc(c,kw)
        if c: self.children = self.children+c
        if kw: self.attrs = {**self.attrs, **kw}
        return self.changed()

    def set(self, *c, **kw):
        "Set children and/or attributes (chainable)"
        c,kw = _preproc(c,kw)
        if c: self.children = c
        if kw:
            self.attrs = {k:v for k,v in self.attrs.items() if k in ('id','name')}
            self.attrs = {**self.attrs, **kw}
        return self.changed()

### 1. `__init__` Method

```python
def __init__(self, tag:str, cs:tuple, attrs:dict=None, void_=False, **kwargs):
    assert isinstance(cs, tuple)
    self.tag,self.children,self.attrs,self.void_ = tag,cs,attrs,void_
    self.listeners_ = []
```

**Explanation:**
- This initializes a new FT (Fast Tag) object
- Parameters:
  - `tag`: String representing the HTML/XML tag name
  - `cs`: Tuple of children elements
  - `attrs`: Dictionary of attributes (defaults to None)
  - `void_`: Boolean indicating if this is a void element (elements that don't need closing tags like `<img>`)
- The method asserts that `cs` must be a tuple to ensure type safety
- It then assigns the parameters to instance attributes
- Finally, it initializes an empty list of listeners (callbacks for change notifications)

**Example:**
```python
# Creating a div with "Hello" as child and class="container" as attribute
div = FT("div", ("Hello",), {"class": "container"})
```

### 2. `tag` Attribute

This is an instance attribute set in `__init__`. It stores the HTML/XML tag name as a string.

**Example:**
```python
div = FT("div", (), {})
print(div.tag)  # Outputs: "div"
```

### 3. `children` Attribute

This is an instance attribute set in `__init__`. It stores the child elements as a tuple.

**Example:**
```python
div = FT("div", ("Hello", "World"), {})
print(div.children)  # Outputs: ('Hello', 'World')
```

### 4. `attrs` Attribute

This is an instance attribute set in `__init__`. It stores the HTML/XML attributes as a dictionary.

**Example:**
```python
div = FT("div", (), {"class": "container", "id": "main"})
print(div.attrs)  # Outputs: {'class': 'container', 'id': 'main'}
```

### 5. `void_` Attribute

This is an instance attribute set in `__init__`. It's a boolean indicating if the element is a void element (doesn't need a closing tag).

**Example:**
```python
img = FT("img", (), {"src": "image.jpg"}, void_=True)
print(img.void_)  # Outputs: True
```

### 6. `listeners_` Attribute

This is an instance attribute initialized in `__init__` as an empty list. It stores callback functions to be called when the element changes.

**Example:**
```python
div = FT("div", (), {})
print(div.listeners_)  # Outputs: []
```

### 7. `on` Method

```python
def on(self, f): self.listeners_.append(f)
```

**Explanation:**
- This method adds a callback function to the listeners list
- Parameter `f`: The callback function to add
- The function will be called whenever the element changes
- Used for implementing reactive behavior

**Example:**
```python
div = FT("div", (), {})
def notify_change(element):
    print(f"Element {element.tag} changed")
div.on(notify_change)
```

### 8. `changed` Method

```python
def changed(self):
    [f(self) for f in self.listeners_]
    return self
```

**Explanation:**
- This method calls all registered listener callbacks with the element as argument
- It uses a list comprehension to iterate through all listeners and call each one
- Returns `self` to allow method chaining
- Called internally whenever the element is modified

**Example:**
```python
div = FT("div", (), {})
div.on(lambda el: print(f"{el.tag} changed"))
div.changed()  # Outputs: "div changed"
```

### 9. `__setattr__` Method

```python
def __setattr__(self, k, v):
    if len(k)>1 and k.startswith('__') or k[-1]=='_' or k in ('tag','children','attrs','void_'): 
        return super().__setattr__(k,v)
    self.attrs[_fix_k(k)] = v
    self.changed()
```

**Explanation:**
- Custom attribute setter that handles special cases for attributes
- If the attribute name:
  - Starts with double underscore (`__`) and is longer than 1 character
  - Ends with underscore (`_`)
  - Is one of the core attributes ('tag', 'children', 'attrs', 'void_')
- Then it behaves like a normal attribute setter
- Otherwise, it treats the attribute as an HTML attribute and adds it to `self.attrs`
- It uses `_fix_k()` to transform the attribute name
- Calls `self.changed()` to notify listeners of the change

**Example:**
```python
div = FT("div", (), {})
div.class_ = "container"  # Normal Python attribute (ends with _)
div.id = "main"          # Treated as HTML attribute
print(div.attrs)         # Outputs: {'id': 'main'}
```

### 10. `__getattr__` Method

```python
def __getattr__(self, k):
    if k.startswith('__'): raise AttributeError(k)
    return self.get(k)
```

**Explanation:**
- Custom attribute getter for HTML attributes
- If the attribute name starts with double underscore (`__`), it raises an AttributeError
- Otherwise, it calls `self.get(k)` to retrieve the attribute from `self.attrs`
- This allows accessing HTML attributes directly as Python attributes

**Example:**
```python
div = FT("div", (), {"class": "container"})
print(div.class_)  # This would call __getattr__ and return "container"
```

I'll continue with detailed explanations for the remaining methods in the FT class:

### 11. `list` Property

```python
@property
def list(self): return [self.tag, self.children, self.attrs]
```

**Explanation:**
- This is a property that returns a list representation of the element
- Returns a list containing three items: the tag name, the children tuple, and the attributes dictionary
- This provides a convenient way to access the core components of the element in a list format
- Used internally by some functions that need these components separately

**Example:**
```python
div = FT("div", ("Hello",), {"class": "container"})
tag, children, attrs = div.list
print(tag)       # Outputs: "div"
print(children)  # Outputs: ('Hello',)
print(attrs)     # Outputs: {'class': 'container'}
```

### 12. `get` Method

```python
def get(self, k, default=None): return self.attrs.get(_fix_k(k), default)
```

**Explanation:**
- Retrieves an attribute value from the `attrs` dictionary
- Parameters:
  - `k`: The attribute name to retrieve
  - `default`: The value to return if the attribute doesn't exist (defaults to None)
- Uses `_fix_k()` to transform the attribute name before lookup
- Provides a safe way to access attributes with a fallback default value

**Example:**
```python
div = FT("div", (), {"class": "container"})
print(div.get("class"))          # Outputs: "container"
print(div.get("id", "default"))  # Outputs: "default" (as "id" doesn't exist)
```

### 13. `__repr__` Method

```python
def __repr__(self): return f'{self.tag}({self.children},{self.attrs})'
```

**Explanation:**
- Provides a string representation of the FT object
- Returns a string in the format: `tag(children,attrs)`
- Used when printing the object or when displaying it in interactive environments
- Helpful for debugging and inspecting FT objects

**Example:**
```python
div = FT("div", ("Hello",), {"class": "container"})
print(div)  # Outputs something like: div(('Hello',),{'class': 'container'})
```

### 14. `__iter__` Method

```python
def __iter__(self): return iter(self.children)
```

**Explanation:**
- Makes the FT object iterable by returning an iterator over its children
- Allows using FT objects in for loops to iterate through their children
- Enables unpacking of the children using the * operator

**Example:**
```python
div = FT("div", ("Hello", "World"), {})
for child in div:
    print(child)  # Outputs: "Hello" then "World"
```

### 15. `__getitem__` Method

```python
def __getitem__(self, idx): return self.children[idx]
```

**Explanation:**
- Allows accessing children by index using square bracket notation
- Simply delegates to the indexing of the `children` tuple
- Makes FT objects behave like sequences for child access

**Example:**
```python
div = FT("div", ("Hello", "World"), {})
print(div[0])  # Outputs: "Hello"
print(div[1])  # Outputs: "World"
```

### 16. `__setitem__` Method

```python
def __setitem__(self, i, o):
    self.children = self.children[:i] + (o,) + self.children[i+1:]
    self.changed()
```

**Explanation:**
- Allows setting a child at a specific index using square bracket notation
- Creates a new tuple by concatenating:
  - The slice of children before index `i`
  - A tuple containing just the new child `o`
  - The slice of children after index `i`
- Calls `self.changed()` to notify listeners of the change
- Note that this creates a new tuple since tuples are immutable

**Example:**
```python
div = FT("div", ("Hello", "World"), {})
div[1] = "Python"
print(div.children)  # Outputs: ('Hello', 'Python')
```

### 17. `__call__` Method

```python
def __call__(self, *c, **kw):
    c,kw = _preproc(c,kw)
    if c: self.children = self.children+c
    if kw: self.attrs = {**self.attrs, **kw}
    return self.changed()
```

**Explanation:**
- Makes the FT object callable, allowing to add children and attributes after creation
- Processes the arguments using `_preproc`
- If children `c` are provided, appends them to the existing children
- If keyword arguments `kw` are provided, updates the attributes dictionary
- Calls `self.changed()` to notify listeners and returns self for chaining
- This enables a fluent API style where you can add to elements after creation

**Example:**
```python
div = FT("div", (), {"class": "container"})
div("Hello", id="main")
# Now div has child "Hello" and attributes class="container" and id="main"
```

### 18. `set` Method

```python
def set(self, *c, **kw):
    "Set children and/or attributes (chainable)"
    c,kw = _preproc(c,kw)
    if c: self.children = c
    if kw:
        self.attrs = {k:v for k,v in self.attrs.items() if k in ('id','name')}
        self.attrs = {**self.attrs, **kw}
    return self.changed()
```

**Explanation:**
- Similar to `__call__`, but replaces children and attributes instead of adding to them
- Processes the arguments using `_preproc`
- If children `c` are provided, replaces all existing children
- If keyword arguments `kw` are provided:
  - Preserves only 'id' and 'name' attributes from the existing attributes
  - Updates with the new attributes
- Calls `self.changed()` to notify listeners and returns self for chaining
- Useful when you want to completely reset an element's content while preserving its identity

**Example:**
```python
div = FT("div", ("Old content",), {"class": "old", "id": "main"})
div.set("New content", class_="new")
# Now div has only child "New content" and attributes id="main" and class="new"
```

The methods of the FT class work together to create a flexible and powerful API for building and manipulating HTML/XML elements in Python. The combination of Python's special methods (`__getattr__`, `__setattr__`, `__call__`, etc.) with carefully designed helper functions creates an intuitive interface that feels natural to use.

In [ ]:
#| export
def ft(tag:str, *c, void_:bool=False, attrmap:callable=attrmap, valmap:callable=valmap, ft_cls=FT, **kw):
    "Create an `FT` structure for `to_xml()`"
    return ft_cls(tag.lower(),*_preproc(c,kw,attrmap=attrmap, valmap=valmap), void_=void_)

In [ ]:
#| export
voids = set('area base br col command embed hr img input keygen link meta param source track wbr !doctype'.split())
_g = globals()
_all_ = ['Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'A', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(ft, o.lower(), void_=o.lower() in voids)

The variable `_g = globals()` is defined in the source code to get a reference to the global namespace dictionary. This is used later in the code to dynamically create HTML tag functions. Specifically, it's used in this part:

```python
for o in _all_: _g[o] = partial(ft, o.lower(), void_=o.lower() in voids)
```

This is a technique to programmatically add functions to the module's global namespace, allowing users to access HTML tag functions like `Div()`, `P()`, etc. directly.

The main HTML tags are exported as `ft` partials.

Attributes are passed as keywords. Use 'klass' and 'fr' instead of 'class' and 'for', to avoid Python reserved word clashes.

In [ ]:
#| export
def Html(*c, doctype=True, **kwargs)->FT:
    "An HTML tag, optionally preceeded by `!DOCTYPE HTML`"
    res = ft('html', *c, **kwargs)
    if not doctype: return res
    return (ft('!DOCTYPE', html=True, void_=True), res)

In [ ]:
samp = Html(
    Head(Title('Some page')),
    Body(Div('Some text\nanother line', (Input(name="jph's"), Img(src="filename", data=1)),
             cls=['myclass', 'another'],
             style={'padding':1, 'margin':2}))
)
pprint(samp)

(!doctype((),{'html': True}),
 html((head((title(('Some page',),{}),),{}), body((div(('Some text\nanother line', input((),{'name': "jph's"}), img((),{'src': 'filename', 'data': 1})),{'class': 'myclass another', 'style': 'padding:1; margin:2'}),),{})),{}))


In [ ]:
elem = P('Some text', id="myid")
print(elem.tag)
print(elem.children)
print(elem.attrs)

p
('Some text',)
{'id': 'myid'}


You can get and set attrs directly:

In [ ]:
elem.id = 'newid'
print(elem.id, elem.get('id'), elem.get('foo', 'missing'))
elem

newid newid missing


p(('Some text',),{'id': 'newid'})

In [ ]:
#| export
class Safe(str):
    def __html__(self): return self

## Conversion to XML/HTML

In [ ]:
#| export
def _escape(s): return '' if s is None else s.__html__() if hasattr(s, '__html__') else escape(s) if isinstance(s, str) else s
def _noescape(s): return '' if s is None else s.__html__() if hasattr(s, '__html__') else s

In [ ]:
#| export
def _to_attr(k,v):
    if isinstance(v,bool):
        if v==True : return str(k)
        if v==False: return ''
    if isinstance(v,str): v = escape(v, quote=False)
    elif isinstance(v, Mapping): v = json.dumps(v)
    else: v = str(v)
    qt = '"'
    if qt in v:
        qt = "'"
        if "'" in v: v = v.replace("'", "&#39;")
    return f'{k}={qt}{v}{qt}'

In [ ]:
#| export
_block_tags = {'div', 'p', 'ul', 'ol', 'li', 'table', 'thead', 'tbody', 'tfoot',
               'html', 'head', 'body', 'meta', 'title', '!doctype', 'input', 'script', 'link', 'style',
               'tr', 'th', 'td', 'section', 'article', 'nav', 'aside', 'header',
               'footer', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'blockquote'}
_inline_tags = {'a', 'span', 'b', 'i', 'u', 'em', 'strong', 'img', 'br', 'small',
                'big', 'sub', 'sup', 'label', 'input', 'select', 'option'}

def _is_whitespace_significant(elm):
    return elm.tag in {'pre', 'code', 'textarea', 'script'} or elm.get('contenteditable') == 'true'

In [ ]:
#| export
def _to_xml(elm, lvl=0, indent=True, do_escape=True):
    "Convert `FT` element tree into an XML string"
    esc_fn = _escape if do_escape else _noescape
    if elm is None: return ''
    if hasattr(elm, '__ft__'): elm = elm.__ft__()
    if isinstance(elm, tuple):
        return ''.join(_to_xml(o, lvl=lvl, indent=indent, do_escape=do_escape) for o in elm)
    if isinstance(elm, bytes): return elm.decode('utf-8')
    if not isinstance(elm, FT): return f'{esc_fn(elm)}'

    tag, cs, attrs = elm.list
    is_void = getattr(elm, 'void_', False)
    is_block = tag in _block_tags
    if _is_whitespace_significant(elm): indent = False

    sp,nl = (' ' * lvl,'\n') if indent and is_block else ('','')
    nl_end = nl

    stag = tag
    if attrs:
        sattrs = ' '.join(_to_attr(k, v) for k, v in attrs.items() if v not in (False, None, '') and (k=='_' or k[-1]!='_'))
        if sattrs: stag += f' {sattrs}'

    cltag = '' if is_void else f'</{tag}>'

    if not cs:
        if is_void: return f'{sp}<{stag}>{nl_end}'
        else: return f'{sp}<{stag}>{cltag}{nl_end}'
    if len(cs) == 1 and not isinstance(cs[0], (list, tuple, FT)) and not hasattr(cs[0], '__ft__'):
        content = esc_fn(cs[0])
        return f'{sp}<{stag}>{content}{cltag}{nl_end}'

    res = f'{sp}<{stag}>{nl}'
    for c in cs:
        res += _to_xml(c, lvl=lvl+2 if indent else 0, indent=indent, do_escape=do_escape)
    if not is_void: res += f'{sp}{cltag}{nl_end}'
    return Safe(res)

In [ ]:
#| export
def to_xml(elm, lvl=0, indent=True, do_escape=True):
    "Convert `ft` element tree into an XML string"
    return Safe(_to_xml(elm, lvl, indent, do_escape=do_escape))

FT.__html__ = to_xml

In [ ]:
#| hide
test_eq(to_xml(Div("Hello")), '<div>Hello</div>\n')
test_eq(to_xml(P("Text", Class="test")), '<p class="test">Text</p>\n')
test_eq(to_xml(Div(P("Nested"))), '<div>\n  <p>Nested</p>\n</div>\n')
test_eq(to_xml(Pre("  Whitespace\n  Significant  ")), '<pre>  Whitespace\n  Significant  </pre>')
test_eq(to_xml(Img(src="image.jpg")), '<img src="image.jpg">')
test_eq(to_xml(Div("Text", contenteditable="true")), '<div contenteditable="true">Text</div>')
test_eq(to_xml(None), '')
test_eq(to_xml(("Text", P("Paragraph"))), 'Text<p>Paragraph</p>\n')
test_eq(to_xml(b"Bytes"), 'Bytes')
test_eq(to_xml(Div(P("Text"), B("Bold")), indent=False), '<div><p>Text</p><b>Bold</b></div>')
test_eq(to_xml(Div("<script>alert('XSS')</script>"), do_escape=True),
        '<div>&lt;script&gt;alert(&#x27;XSS&#x27;)&lt;/script&gt;</div>\n')
test_eq(to_xml(Div("<script>alert('XSS')</script>"), do_escape=False),
        "<div><script>alert('XSS')</script></div>\n")
test_eq(to_xml(Div(foo=False), indent=False), '<div></div>')

In [ ]:
#| hide
test_eq(to_xml(B('Bold Text')), '<b>Bold Text</b>')
test_eq(to_xml(Div(P('Paragraph Text'))), '<div>\n  <p>Paragraph Text</p>\n</div>\n')
test_eq(to_xml(Pre('   Preformatted\n   Text')), '<pre>   Preformatted\n   Text</pre>')
editable_div = Div('Editable Content', contenteditable='true')
test_eq(to_xml(editable_div), '<div contenteditable="true">Editable Content</div>')
test_eq(to_xml(Div(Span('Inline Text'), P('Paragraph'))),
        '<div>\n<span>Inline Text</span>  <p>Paragraph</p>\n</div>\n')
test_eq(to_xml(Br()), '<br>')
test_eq(to_xml(P(None)), '<p></p>\n')
test_eq(to_xml(Div()), '<div></div>\n')
test_eq(to_xml(Input(type='text', disabled=True)), '<input type="text" disabled>\n')
special_attr_tag = Div(id='main"div', data_info="Some 'info'")
expected_special_attr = "<div id='main\"div' data-info=\"Some 'info'\"></div>\n"
test_eq(to_xml(special_attr_tag), expected_special_attr)

In [ ]:
h = to_xml(samp, do_escape=False)
print(h)

<!doctype html>
<html>
  <head>
    <title>Some page</title>
  </head>
  <body>
    <div class="myclass another" style="padding:1; margin:2">
Some text
another line      <input name="jph's">
<img src="filename" data="1">    </div>
  </body>
</html>



In [ ]:
class PageTitle:
    def __ft__(self): return H1("Hello")

class HomePage:
    def __ft__(self): return Div(PageTitle(), Div('hello'))

h = to_xml(Div(HomePage()))
expected_output = """<div>
  <div>
    <h1>Hello</h1>
    <div>hello</div>
  </div>
</div>
"""
assert h == expected_output

In [ ]:
print(h)

<div>
  <div>
    <h1>Hello</h1>
    <div>hello</div>
  </div>
</div>



In [ ]:
h = to_xml(samp, indent=False)
print(h)

<!doctype html><html><head><title>Some page</title></head><body><div class="myclass another" style="padding:1; margin:2">Some text
another line<input name="jph's"><img src="filename" data="1"></div></body></html>


Interoperability both directions with Django and Jinja using the [__html__() protocol](https://jinja.palletsprojects.com/en/3.1.x/templates/#jinja-filters.escape):

In [ ]:
def _esc(s): return s.__html__() if hasattr(s, '__html__') else Safe(escape(s))

r = Safe('<b>Hello from Django</b>')
print(to_xml(Div(r)))
print(_esc(Div(P('Hello from fastcore <3'))))

<div><b>Hello from Django</b></div>

<div>
  <p>Hello from fastcore &lt;3</p>
</div>



## Display

In [ ]:
#| export
def highlight(s, lang='html'):
    "Markdown to syntax-highlight `s` in language `lang`"
    return f'```{lang}\n{to_xml(s)}\n```'

In [ ]:
#| export
def showtags(s):
    return f"""<code><pre>
{escape(to_xml(s))}
</code></pre>"""

FT._repr_markdown_ = highlight

You can also reorder the children to come *after* the attrs, if you use this alternative syntax for `FT` where the children are in a second pair of `()` (behind the scenes this is because `FT` implements `__call__` to add children).

In [ ]:
Body(klass='myclass')(
    Div(style='padding:3px')(
        'Some text 1<2',
        I(spurious=True)('in italics'),
        Input(name='me'),
        Img(src="filename", data=1)
    )
)

```html
<body class="myclass">
  <div style="padding:3px">
Some text 1&lt;2<i spurious>in italics</i>    <input name="me">
<img src="filename" data="1">  </div>
</body>

```

In [ ]:
#| export
def __getattr__(tag):
    if tag.startswith('_') or tag[0].islower(): raise AttributeError
    tag = _fix_k(tag)
    def _f(*c, target_id=None, **kwargs): return ft(tag, *c, target_id=target_id, **kwargs)
    return _f

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()